In [69]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.utils as vutils



In [49]:
transforms=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

In [50]:
train_data=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transforms,download=True)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transforms,download=True)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)


In [51]:
image,label=train_data[0]

In [52]:
image.size()

torch.Size([3, 32, 32])

In [53]:
class_names=["plane","car","bird","cat","deer","dog","frog","horse","ship","truck"]

In [54]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,12,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(12,24,5)
        self.fc1=nn.Linear(24*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x




In [55]:
net=NeuralNet()
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [56]:
for epoch in range(30):
    print(f"Training epoch {epoch+1}")

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Loss: {epoch_loss:.4f}")


Training epoch 1
Loss: 2.1805
Training epoch 2
Loss: 1.7437
Training epoch 3
Loss: 1.4980
Training epoch 4
Loss: 1.3706
Training epoch 5
Loss: 1.2748
Training epoch 6
Loss: 1.1915
Training epoch 7
Loss: 1.1188
Training epoch 8
Loss: 1.0622
Training epoch 9
Loss: 1.0097
Training epoch 10
Loss: 0.9654
Training epoch 11
Loss: 0.9265
Training epoch 12
Loss: 0.8869
Training epoch 13
Loss: 0.8491
Training epoch 14
Loss: 0.8176
Training epoch 15
Loss: 0.7906
Training epoch 16
Loss: 0.7620
Training epoch 17
Loss: 0.7306
Training epoch 18
Loss: 0.7095
Training epoch 19
Loss: 0.6823
Training epoch 20
Loss: 0.6597
Training epoch 21
Loss: 0.6368
Training epoch 22
Loss: 0.6149
Training epoch 23
Loss: 0.5942
Training epoch 24
Loss: 0.5715
Training epoch 25
Loss: 0.5539
Training epoch 26
Loss: 0.5341
Training epoch 27
Loss: 0.5129
Training epoch 28
Loss: 0.4929
Training epoch 29
Loss: 0.4764
Training epoch 30
Loss: 0.4542


In [60]:
torch.save(net.state_dict(),'trained_net.pth')


In [61]:
net=NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [62]:
correct=0
total=0
net.eval()
with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=net(images)
    _, predicted = torch.max(outputs, 1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
accuracy=100*correct/total
print(f'Accuracy: {accuracy}%')

Accuracy: 69.98%


In [74]:
from google.colab import files

uploaded = files.upload()


Saving t22.jpg to t22.jpg


In [72]:
from google.colab import files

uploaded = files.upload()


Saving t12.jpg to t12.jpg


In [75]:
new_transform=torchvision.transforms.Compose([
    torchvision.transforms.Resize((32,32)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])

])

def load_image(image_path):
  image=Image.open(image_path)
  image=new_transform(image)
  image=image.unsqueeze(0)
  return image

image_paths=['t22.jpg','t12.jpg']
images=[load_image(img) for img in image_paths]

net.eval()

with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')


Prediction: dog
Prediction: ship
